### Passo 1 - Importar Arquivos e Bibliotecas

In [1]:
import pandas as pd
import smtplib
import email.message
import mimetypes 

from pathlib import Path

In [6]:

sales_df = pd.read_excel('Bases de Dados/Vendas.xlsx')
emails_df = pd.read_csv('Bases de Dados/Emails.csv')
stores_df = pd.read_csv('Bases de Dados/Lojas.csv', sep=';', encoding='latin-1')
emails_df = emails_df.drop(columns=['Unnamed: 0.1', 'Unnamed: 0'], axis=1)

display(sales_df.head())
display(emails_df.head())
display(stores_df.head())

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final
0,1,2019-01-01,1,Sapato Estampa,1,358,358
1,1,2019-01-01,1,Camiseta,2,180,360
2,1,2019-01-01,1,Sapato Xadrez,1,368,368
3,2,2019-01-02,3,Relógio,3,200,600
4,2,2019-01-02,3,Chinelo Liso,1,71,71


,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,cezarfrancisco63+helena@gmail.com
1,Shopping Midway Mall,Alice,cezarfrancisco63+alice@gmail.com
2,Norte Shopping,Laura,cezarfrancisco63+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,cezarfrancisco63+manuela@gmail.com
4,Shopping União de Osasco,Valentina,cezarfrancisco63+valentina@gmail.com


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco


In [9]:
display(emails_df)
display(stores_df)


,Loja,Gerente,E-mail
0,Iguatemi Esplanada,Helena,cezarfrancisco63+helena@gmail.com
1,Shopping Midway Mall,Alice,cezarfrancisco63+alice@gmail.com
2,Norte Shopping,Laura,cezarfrancisco63+laura@gmail.com
3,Shopping Iguatemi Fortaleza,Manuela,cezarfrancisco63+manuela@gmail.com
4,Shopping União de Osasco,Valentina,cezarfrancisco63+valentina@gmail.com
5,Shopping Center Interlagos,Sophia,cezarfrancisco63+sophia@gmail.com
6,Rio Mar Recife,Isabella,cezarfrancisco63+isabella@gmail.com
7,Salvador Shopping,Heloisa,cezarfrancisco63+heloisa@gmail.com
8,Rio Mar Shopping Fortaleza,Luiza,cezarfrancisco63+luiza@gmail.com
9,Shopping Center Leste Aricanduva,Julia,cezarfrancisco63+julia@gmail.com


,ID Loja,Loja
0,1,Iguatemi Esplanada
1,2,Shopping Midway Mall
2,3,Norte Shopping
3,4,Shopping Iguatemi Fortaleza
4,5,Shopping União de Osasco
5,6,Shopping Center Interlagos
6,7,Rio Mar Recife
7,8,Salvador Shopping
8,9,Rio Mar Shopping Fortaleza
9,10,Shopping Center Leste Aricanduva


### Passo 2 - Definir Criar uma Tabela para cada Loja e Definir o dia do Indicador

In [25]:
sales_df = sales_df.merge(stores_df, on='ID Loja')
display(sales_df)

,Código Venda,Data,ID Loja,Produto,Quantidade,Valor Unitário,Valor Final,Loja
0,1,2019-01-01,1,Sapato Estampa,1,358,358,Iguatemi Esplanada
1,1,2019-01-01,1,Camiseta,2,180,360,Iguatemi Esplanada
2,1,2019-01-01,1,Sapato Xadrez,1,368,368,Iguatemi Esplanada
3,21,2019-01-02,1,Camisa Gola V Listrado,2,116,232,Iguatemi Esplanada
4,34,2019-01-02,1,Sapato Listrado,1,363,363,Iguatemi Esplanada
...,...,...,...,...,...,...,...,...
100994,69846,2019-12-25,8,Short Estampa,2,96,192,Salvador Shopping
100995,69846,2019-12-25,8,Tênis Estampa,5,256,1280,Salvador Shopping
100996,69850,2019-12-25,8,Calça Estampa,4,177,708,Salvador Shopping
100997,69972,2019-12-26,8,Terno Liso,3,720,2160,Salvador Shopping


In [28]:
last_date = sales_df['Data'].max()
print(last_date)

2019-12-26 00:00:00


### Passo 3 - Salvar a planilha na pasta de backup

In [26]:
dict_stores = {}
for store in stores_df['Loja']:
    dict_stores[store] = sales_df.loc[sales_df['Loja']==store, :]



In [9]:
backup_path = Path('Stores Files Backup')

folders_backup = backup_path.iterdir()
names_list_backup = [folder.name for folder in folders_backup]

for store in stores_df['Loja']:

    # Checks if the folder does not exist and makes it
    if store not in names_list_backup:
        Path(backup_path/store).mkdir()
    
    # Saves the files in the folder
    file_name = f'{last_date.day}_{last_date.month}_{store}.csv'
    dict_stores[store].to_csv(backup_path/store/file_name)


In [13]:
backup_path = Path('Stores Files Backup')

folders_backup = backup_path.iterdir()
for folder in folders_backup:
    print(folder.name)


Novo Shopping Ribeirão Preto
Rio Mar Recife
Shopping Center Interlagos
Ribeirão Shopping
Norte Shopping
Iguatemi Esplanada
26_12_Annual Ranking.csv
Palladium Shopping Curitiba
Shopping Morumbi
Shopping Barra
Shopping Vila Velha
Shopping Center Leste Aricanduva
Bourbon Shopping SP
Parque Dom Pedro Shopping
Iguatemi Campinas
Passei das Águas Shopping
Rio Mar Shopping Fortaleza
Shopping Eldorado
Center Shopping Uberlândia
Shopping Midway Mall
Shopping União de Osasco
Shopping SP Market
Shopping Iguatemi Fortaleza
26_12_Days Ranking.csv
Shopping Recife
Salvador Shopping
Shopping Ibirapuera


### Passo 4 - Calcular o indicador para 1 loja

In [10]:
#definição de metas

target_billing_day = 1000
target_billing_year = 1650000
product_quantity_goal_day = 4
product_quantity_goal_year = 120
target_average_sales_day = 500
target_average_sales_year = 500

In [32]:

store_sales = dict_stores[store]
store_sales_date = store_sales.loc[store_sales['Data']==last_date, :]
# store_sales = store_sales.min()
print(store_sales.info)

<bound method DataFrame.info of        Código Venda       Data  ID Loja             Produto  Quantidade  \
12277             5 2019-01-02       25               Calça           1   
12278            52 2019-01-02       25      Gorro Listrado           4   
12279           100 2019-01-02       25  Camisa Gola V Liso           3   
12280           100 2019-01-02       25        Cinto Xadrez           4   
12281           132 2019-01-02       25         Terno Linho           2   
...             ...        ...      ...                 ...         ...   
16234         69908 2019-12-25       25           Meia Liso           2   
16235         69908 2019-12-25       25              Camisa           1   
16236         69913 2019-12-25       25               Gorro           1   
16237         69913 2019-12-25       25         Cinto Linho           1   
16238         69950 2019-12-25       25         Cinto Linho           1   

       Valor Unitário  Valor Final            Loja  
12277         

In [11]:
# for store in dict_stores:

#     store_sales = dict_stores[store]
#     store_sales_date = store_sales.loc[store_sales['Data']==last_date, :]

#     # Store Billing
#     year_store_billing = store_sales['Valor Final'].sum()
#     day_store_billing = store_sales_date['Valor Final'].sum()

#     # How many different products were sales
#     products_sold_year = len(store_sales['Produto'].unique())
#     products_sold_day = len(store_sales_date['Produto'].unique())

#     # average sale value on the year
#     sales_value_year = store_sales.groupby('Código Venda').sum(['Valor Final'])
#     mean_year_sales_value = sales_value_year['Valor Final'].mean()
#     # average sale value on the day
#     value_sales_day = store_sales_date.groupby('Código Venda').sum(['Valor Final'])
#     mean_day_sales_value = value_sales_day['Valor Final'].mean()

#     from_email = '<e-mail>'
#     password = '<password>'
#     to_email = emails_df.loc[emails_df['Loja']==store, 'E-mail'].values[0]


#     nome = emails_df.loc[emails_df['Loja']==store, 'Gerente'].values[0]

#     if day_store_billing >= target_average_sales_day: cor_fat_dia = 'green' 
#     else: cor_fat_dia = 'red'

#     if year_store_billing >= target_billing_year: cor_fat_ano = 'green'
#     else: cor_fat_ano = 'red'

#     if products_sold_day >= product_quantity_goal_day: cor_qtde_dia = 'green'
#     else: cor_qtde_dia = 'red'

#     if products_sold_year >= product_quantity_goal_year: cor_qtde_ano = 'green'
#     else: cor_qtde_ano = 'red'

#     if mean_day_sales_value >= target_average_sales_day: cor_ticket_dia = 'green'
#     else: cor_ticket_dia = 'red'

#     if mean_year_sales_value >= target_average_sales_year: cor_ticket_ano = 'green'
#     else: cor_ticket_ano = 'red'

#     corpo_email = f"""
#         <p>Bom dia, {nome}</p>

#         <p>O resultado de ontem <strong>({last_date.day}/{last_date.month})</strong> da <strong>Loja {store}</strong> foi:</p>

#         <table>
#           <tr>
#             <th>Indicador</th>
#             <th>Valor Dia</th>
#             <th>Meta Dia</th>
#             <th>Cenário Dia</th>
#           </tr>
#           <tr>
#             <td>Faturamento</td>
#             <td style="text-align: center">R${day_store_billing:.2f}</td>
#             <td style="text-align: center">R${target_billing_day:.2f}</td>
#             <td style="text-align: center"><font color="{cor_fat_dia}">◙</font></td>
#           </tr>
#           <tr>
#             <td>Diversidade de Produtos</td>
#             <td style="text-align: center">{products_sold_day}</td>
#             <td style="text-align: center">{product_quantity_goal_day}</td>
#             <td style="text-align: center"><font color="{cor_qtde_dia}">◙</font></td>
#           </tr>
#           <tr>
#             <td>Ticket Médio</td>
#             <td style="text-align: center">R${mean_day_sales_value:.2f}</td>
#             <td style="text-align: center">R${target_average_sales_day:.2f}</td>
#             <td style="text-align: center"><font color="{cor_ticket_dia}">◙</font></td>
#           </tr>
#         </table>
#         <br>
#         <table>
#           <tr>
#             <th>Indicador</th>
#             <th>Valor Ano</th>
#             <th>Meta Ano</th>
#             <th>Cenário Ano</th>
#           </tr>
#           <tr>
#             <td>Faturamento</td>
#             <td style="text-align: center">R${year_store_billing:.2f}</td>
#             <td style="text-align: center">R${target_billing_year:.2f}</td>
#             <td style="text-align: center"><font color="{cor_fat_ano}">◙</font></td>
#           </tr>
#           <tr>
#             <td>Diversidade de Produtos</td>
#             <td style="text-align: center">{products_sold_year}</td>
#             <td style="text-align: center">{product_quantity_goal_year}</td>
#             <td style="text-align: center"><font color="{cor_qtde_ano}">◙</font></td>
#           </tr>
#           <tr>
#             <td>Ticket Médio</td>
#             <td style="text-align: center">R${mean_year_sales_value:.2f}</td>
#             <td style="text-align: center">R${target_average_sales_year:.2f}</td>
#             <td style="text-align: center"><font color="{cor_ticket_ano}">◙</font></td>
#           </tr>
#         </table>

#         <p>Segue <strong>em anexo</strong> a planilha com todos os dados para mais detalhes.</p>

#         <p>Qualquer dúvida estou à disposição.</p>
#         <p>Francsico</p>
#         """

#     msg = email.message.EmailMessage()
#     msg['Subject'] = f'OnePage Dia {last_date.day}/{last_date.month} - Loja {store}'
#     msg['From'] = from_email
#     msg['To'] = to_email
#     msg.add_header('Content-Type', 'text/html')
#     msg.set_payload(corpo_email)

#     mime_type, _ = mimetypes.guess_type(f'{store}.csv')
#     mime_type, mime_subtype = mime_type.split('/')

#     att  = Path.cwd() / backup_path / store / f'{last_date.day}_{last_date.month}_{store}.csv'


#     with open(att, 'rb') as file:
#         msg.add_attachment(file.read(),
#         maintype=mime_type,
#         subtype=mime_subtype,
#         filename=f'{store}.csv')



#     server = smtplib.SMTP('smtp.gmail.com: 587')
#     server.starttls()
#     # Login Credentials for sending the mail
#     server.login(msg['From'], password)
#     server.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))


#     print(f'Email da loja {store} enviado!')

Email da loja Iguatemi Esplanada enviado!
Email da loja Shopping Midway Mall enviado!
Email da loja Norte Shopping enviado!
Email da loja Shopping Iguatemi Fortaleza enviado!
Email da loja Shopping União de Osasco enviado!
Email da loja Shopping Center Interlagos enviado!
Email da loja Rio Mar Recife enviado!
Email da loja Salvador Shopping enviado!
Email da loja Rio Mar Shopping Fortaleza enviado!
Email da loja Shopping Center Leste Aricanduva enviado!
Email da loja Ribeirão Shopping enviado!
Email da loja Shopping Morumbi enviado!
Email da loja Parque Dom Pedro Shopping enviado!
Email da loja Bourbon Shopping SP enviado!
Email da loja Palladium Shopping Curitiba enviado!
Email da loja Passei das Águas Shopping enviado!
Email da loja Center Shopping Uberlândia enviado!
Email da loja Shopping Recife enviado!
Email da loja Shopping Vila Velha enviado!
Email da loja Shopping SP Market enviado!
Email da loja Shopping Eldorado enviado!
Email da loja Shopping Ibirapuera enviado!
Email da lo

### Passo 5 - Enviar por e-mail para o gerente

In [13]:
total_billing = sales_df.groupby('Loja')[['Loja', 'Valor Final']].sum('Valor Final')
annual_billing = total_billing.sort_values(by='Valor Final', ascending=False)

annual_rk_file = f'{last_date.day}_{last_date.month}_Annual Ranking.csv'
annual_billing.to_csv(f'Stores Files Backup/{annual_rk_file}')

store_sales_date = sales_df.loc[sales_df['Data']==last_date, :]
stores_billing_day = store_sales_date.groupby('Loja')[['Loja', 'Valor Final']].sum('Valor Final')
stores_billing_day = stores_billing_day.sort_values(by='Valor Final', ascending=False)

days_rk_file = f"{last_date.day}_{last_date.month}_Days Ranking.csv"
stores_billing_day.to_csv(f'Stores Files Backup/{days_rk_file}')


### Passo 6 - Automatizar todas as lojas

### Passo 7 - Criar ranking para diretoria

### Passo 8 - Enviar e-mail para diretoria

In [44]:
year_best_store = f'R${annual_billing.iloc[0, 0]:,.2f}'.replace(',', '_').replace('.', ',').replace('_', '.')
year_worse_store = f'R${annual_billing.iloc[-1, 0]:,.2f}'.replace(',', '_').replace('.', ',').replace('_', '.')

day_best_store = f'R${stores_billing_day.iloc[0, 0]:,.2f}'.replace(',', '_').replace('.', ',').replace('_', '.')
day_worse_store = f'R${stores_billing_day.iloc[-1, 0]:,.2f}'.replace(',', '_').replace('.', ',').replace('_', '.')

from_email = '<e-mail>'
password = '<password>' 
to_email = emails_df.loc[emails_df['Loja']=='Diretoria', 'E-mail'].values[0]

corpo_email = f"""
<p>Prezados,</p>
<br>
<p>Melhor loja do Dia em Faturamento: Loja {stores_billing_day.index[0]} com Faturamento {day_best_store}</p>
<p>Pior loja do Dia em Faturamento: Loja {stores_billing_day.index[-1]} com Faturamento {day_worse_store}</p>
<br>
<p>Melhor loja do Ano em Faturamento: Loja {annual_billing.index[0]} com Faturamento {year_best_store}</p>
<p>Pior loja do Ano em Faturamento: Loja {annual_billing.index[-1]} com Faturamento {year_worse_store}</p>
<br>
<p>Segue em anexo os rankings do ano e do dia de todas as lojas.</p>
<br>
<p>Qualquer dúvida estou à disposição.</p>
<br>
<p>Francsico</p>
"""

msg = email.message.EmailMessage()
msg['Subject'] = f'Ranking do Dia {last_date.day}/{last_date.month}'
msg['From'] = from_email
msg['To'] = to_email
msg.add_header('Content-Type', 'text/html')
msg.set_payload(corpo_email)

mime_type, _ = mimetypes.guess_type('Days Ranking.csv')
mime_type, mime_subtype = mime_type.split('/')

annual_rk_path  = Path.cwd() / backup_path / f'{last_date.day}_{last_date.month}_Annual Ranking.csv'
days_rk_path  = Path.cwd() / backup_path / f'{last_date.day}_{last_date.month}_Days Ranking.csv'


with open(annual_rk_path, 'rb') as file:
    msg.add_attachment(file.read(),
    maintype=mime_type,
    subtype=mime_subtype,
    filename='Annual Ranking.csv')

with open(days_rk_path, 'rb') as file:
    msg.add_attachment(file.read(),
    maintype=mime_type,
    subtype=mime_subtype,
    filename='Days Ranking.csv')


server = smtplib.SMTP('smtp.gmail.com: 587')
server.starttls()
# Login Credentials for sending the mail
server.login(msg['From'], password)
server.sendmail(msg['From'], [msg['To']], msg.as_string().encode('utf-8'))


print(f'Email para diretoria enviado!')

Email para diretoria enviado!


In [15]:
days_rk_path  = Path.cwd() / backup_path / f'{last_date.day}_{last_date.month}_Days Ranking.csv'
print(days_rk_path)

/Users/franciscojunior/Downloads/projeto_indicadores/Stores Files Backup/26_12_Days Ranking.csv
